Calculate GINI values with different ratios of instances

In [1]:
from scipy.io import loadmat
from scipy.sparse import issparse
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from Data import VFLDataset
from torch.utils.data import DataLoader
import VFL
import torch
import os
DIR = "Data"

In [54]:
file_name = 'RELATHE.mat'
mat = loadmat(os.path.join(DIR, file_name))
X = mat["X"]
y = mat["Y"]
if issparse(X):
    X = X.todense()
y = y.flatten()
print(file_name, X.shape, y.shape)
y[np.where(y == 1)] = 0
y[np.where(y == 2)] = 1
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
dataset = VFLDataset(data_source=(X, y), 
                    num_clients=9,
                    gini_portion=None,
                    insert_noise=False,
                    test_size=0.2)
train_loader = DataLoader(dataset.train(), batch_size=256, shuffle=False)
val_loader = DataLoader(dataset.valid(), batch_size=1000, shuffle=False)
test_loader = DataLoader(dataset.test(), batch_size=1000, shuffle=False)
input_dim_list = dataset.get_input_dim_list()
output_dim = np.unique(y).size
criterion = torch.nn.CrossEntropyLoss()

RELATHE.mat (1427, 4322) (1427,)
Client 0: Feature Index 0-432
Client 1: Feature Index 433-865
Client 2: Feature Index 866-1297
Client 3: Feature Index 1298-1729
Client 4: Feature Index 1730-2161
Client 5: Feature Index 2162-2593
Client 6: Feature Index 2594-3025
Client 7: Feature Index 3026-3457
Client 8: Feature Index 3458-3889
Server : Feature Index 3890-4321


In [60]:
original_gini_labels = dataset.gini_filter(gini_portion=0.5, 
                                           not_init_features=[], 
                                           sample_proportion=0.0, 
                                           random_seed=0)
similarity = np.zeros(5)

for random_seed in range(5):
    gini_labels = dataset.gini_filter(
                          gini_portion=0.5, 
                          not_init_features=[], 
                          sample_proportion=0.9, 
                          random_seed=random_seed
                          )
    print(np.sum(np.multiply(gini_labels, original_gini_labels)))
    print(np.sum(np.multiply(original_gini_labels, original_gini_labels)))
    similarity[random_seed] = np.sum(np.multiply(gini_labels, original_gini_labels)) / np.sum(np.multiply(original_gini_labels, original_gini_labels))

print(np.average(similarity))


1299.0
2161.0
1288.0
2161.0
1275.0
2161.0
1284.0
2161.0
1279.0
2161.0
0.5946321147616844


In [10]:
print(np.random.choice(10, 0, replace=False))

[]
